# Case Study 6

Take a subset of the data and run the neural net presented in class:
N can be any number greater than 1 million, but less than 10.5 million (8GB Ram recommended for all the data).

data source: https://archive.ics.uci.edu/ml/machine-learning-databases/00280/

Work (10 points each)
 1. Pick 3 or more different architectures (add/subtract layers+neurons) and run the model + score. 
 1. With those same 3 architectures, run the SAME architecture but with 2 different (from sigmoid) activation functions.  Google the Keras documentation for a look at different available activations. 
 1. Take your best model from parts 1&2 and vary the batch size by at least 2 orders of magnitude
 1. Take your best model (score) from parts 1&2 and use 3 different kernel initializers. Use a reasonable batch size.
 1. Take your best results from #3 and try 3 different optimizers. (LMGTFY)
 1. Take all that you’ve learned so far and give your best shot at producing a score. 

Questions to be  answered (These are loaded questions—be warned they are there to test your understanding):
 - 10 points - Q1: What was the effect of adding more layers/neurons.
 - 10 points - Q2: Which parameters gave you the best result and why (in your opinion) did they work.
 - 20 points Q3: For #6, how did you decide that your model was ‘done’

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adamax, Adagrad
from sklearn.metrics import roc_auc_score
import keras
from pprint import pprint
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

Using TensorFlow backend.


In [2]:
N=10500000. #Change this line adjust the number of rows. 
data=pd.read_csv("HIGGS.csv",nrows=N,header=None)

In [6]:
test_data=pd.read_csv("HIGGS.csv",nrows=500000,header=None,skiprows=10500000)

In [7]:
#column names from http://archive.ics.uci.edu/ml/datasets/HIGGS
data.columns=['label','lepton pt','lepton eta','lepton phi','missing energy magnitude','missing energy phi',
              'jet 1 pt','jet 1 eta','jet 1 phi','jet 1 b-tag','jet 2 pt','jet 2 eta','jet 2 phi','jet 2 b-tag',
              'jet 3 pt','jet 3 eta','jet 3 phi','jet 3 b-tag','jet 4 pt','jet 4 eta','jet 4 phi','jet 4 b-tag',
              'm_jj','m_jjj','m_lv','m_jlv','m_bb','m_wbb','m_wwbb']
test_data.columns=['label','lepton pt','lepton eta','lepton phi','missing energy magnitude','missing energy phi',
              'jet 1 pt','jet 1 eta','jet 1 phi','jet 1 b-tag','jet 2 pt','jet 2 eta','jet 2 phi','jet 2 b-tag',
              'jet 3 pt','jet 3 eta','jet 3 phi','jet 3 b-tag','jet 4 pt','jet 4 eta','jet 4 phi','jet 4 b-tag',
              'm_jj','m_jjj','m_lv','m_jlv','m_bb','m_wbb','m_wwbb']

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500000 entries, 0 to 10499999
Data columns (total 29 columns):
label                       float64
lepton pt                   float64
lepton eta                  float64
lepton phi                  float64
missing energy magnitude    float64
missing energy phi          float64
jet 1 pt                    float64
jet 1 eta                   float64
jet 1 phi                   float64
jet 1 b-tag                 float64
jet 2 pt                    float64
jet 2 eta                   float64
jet 2 phi                   float64
jet 2 b-tag                 float64
jet 3 pt                    float64
jet 3 eta                   float64
jet 3 phi                   float64
jet 3 b-tag                 float64
jet 4 pt                    float64
jet 4 eta                   float64
jet 4 phi                   float64
jet 4 b-tag                 float64
m_jj                        float64
m_jjj                       float64
m_lv                 

In [9]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 29 columns):
label                       500000 non-null float64
lepton pt                   500000 non-null float64
lepton eta                  500000 non-null float64
lepton phi                  500000 non-null float64
missing energy magnitude    500000 non-null float64
missing energy phi          500000 non-null float64
jet 1 pt                    500000 non-null float64
jet 1 eta                   500000 non-null float64
jet 1 phi                   500000 non-null float64
jet 1 b-tag                 500000 non-null float64
jet 2 pt                    500000 non-null float64
jet 2 eta                   500000 non-null float64
jet 2 phi                   500000 non-null float64
jet 2 b-tag                 500000 non-null float64
jet 3 pt                    500000 non-null float64
jet 3 eta                   500000 non-null float64
jet 3 phi                   500000 non-null float64
jet 3

In [10]:
colnames=list(data.columns.values)
colnames

['label',
 'lepton pt',
 'lepton eta',
 'lepton phi',
 'missing energy magnitude',
 'missing energy phi',
 'jet 1 pt',
 'jet 1 eta',
 'jet 1 phi',
 'jet 1 b-tag',
 'jet 2 pt',
 'jet 2 eta',
 'jet 2 phi',
 'jet 2 b-tag',
 'jet 3 pt',
 'jet 3 eta',
 'jet 3 phi',
 'jet 3 b-tag',
 'jet 4 pt',
 'jet 4 eta',
 'jet 4 phi',
 'jet 4 b-tag',
 'm_jj',
 'm_jjj',
 'm_lv',
 'm_jlv',
 'm_bb',
 'm_wbb',
 'm_wwbb']

In [13]:
assert not ((data["label"]>0)&(data["label"]<1)).sum()

In [14]:
y = np.array(data.iloc[:,0])
x = np.array(data.iloc[:,1:])

y_test = np.array(test_data.iloc[:,0])
x_test = np.array(test_data.iloc[:,1:])

In [15]:
# the target (y) has a mean of close to .5 meaning it's well balanced between 0 and 1
y.mean()

0.5299634285714285

## Work 1

Pick 3 or more different architectures (add/subtract layers+neurons) and run the model + score.

In [16]:
#Begin 
# 1 layer (prof code)
model1 = Sequential()
model1.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model1.add(Activation('sigmoid'))
model1.add(Dropout(0.10))
model1.add(Dense(50, kernel_initializer='uniform'))
model1.add(Activation('sigmoid'))
model1.add(Dropout(0.10))
model1.add(Dense(1, kernel_initializer='uniform')) 
model1.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model1.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model1.predict(x_test))
#end

Epoch 1/5
10500000/10500000 [==============================] - 37s 4us/step - loss: 0.6554 - acc: 0.5969
Epoch 2/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.6101 - acc: 0.6638
Epoch 3/5
10500000/10500000 [==============================] - 35s 3us/step - loss: 0.5910 - acc: 0.6831
Epoch 4/5
10500000/10500000 [==============================] - 35s 3us/step - loss: 0.5819 - acc: 0.6904
Epoch 5/5
10500000/10500000 [==============================] - 35s 3us/step - loss: 0.5758 - acc: 0.6952


0.7798235132728025

In [17]:
# Begin Attempt 1
# Add a layer (3 hidden layers in total) with 50 neurons
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('sigmoid'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('sigmoid'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('sigmoid'))
model2.add(Dropout(0.10))
model2.add(Dense(1, kernel_initializer='uniform')) 
model2.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 41s 4us/step - loss: 0.6917 - acc: 0.5294
Epoch 2/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.6914 - acc: 0.5300
Epoch 3/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.6475 - acc: 0.6144
Epoch 4/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.6087 - acc: 0.6682
Epoch 5/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.5956 - acc: 0.6825


0.7636162454017719

In [18]:
# Begin Attempt 2
# Use original architecture (2 hidden layers in total) but now set all layers to have 25 neurons
model3 = Sequential()
model3.add(Dense(25, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model3.add(Activation('sigmoid'))
model3.add(Dropout(0.10))
model3.add(Dense(25, kernel_initializer='uniform'))
model3.add(Activation('sigmoid'))
model3.add(Dropout(0.10))
model3.add(Dense(1, kernel_initializer='uniform')) 
model3.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model3.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model3.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model3.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 30s 3us/step - loss: 0.6826 - acc: 0.5485
Epoch 2/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.6203 - acc: 0.6520
Epoch 3/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5960 - acc: 0.6790
Epoch 4/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5852 - acc: 0.6878
Epoch 5/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5783 - acc: 0.6933


0.7771522219704127

In [19]:
# Begin Attempt 3
# Use original architecture (2 hidden layers in total) but now set all layers to have 10 neurons
model4 = Sequential()
model4.add(Dense(10, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model4.add(Activation('sigmoid'))
model4.add(Dropout(0.10))
model4.add(Dense(10, kernel_initializer='uniform'))
model4.add(Activation('sigmoid'))
model4.add(Dropout(0.10))
model4.add(Dense(1, kernel_initializer='uniform')) 
model4.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model4.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model4.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model4.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 26s 2us/step - loss: 0.6915 - acc: 0.5300
Epoch 2/5
10500000/10500000 [==============================] - 26s 2us/step - loss: 0.6438 - acc: 0.6216
Epoch 3/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.6074 - acc: 0.6742
Epoch 4/5
10500000/10500000 [==============================] - 30s 3us/step - loss: 0.5975 - acc: 0.6808
Epoch 5/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5934 - acc: 0.6834


0.7590408618992365

## Work 2
With those same 3 architectures, run the SAME architecture but with 2 different (from sigmoid) activation functions.  Google the Keras documentation for a look at different available activations. 


In [20]:
# Begin Attempt 1
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1, kernel_initializer='uniform')) 
model2.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5655 - acc: 0.6998
Epoch 2/5
10500000/10500000 [==============================] - 38s 4us/step - loss: 0.5306 - acc: 0.7298
Epoch 3/5
10500000/10500000 [==============================] - 38s 4us/step - loss: 0.5238 - acc: 0.7346
Epoch 4/5
10500000/10500000 [==============================] - 38s 4us/step - loss: 0.5210 - acc: 0.7366
Epoch 5/5
10500000/10500000 [==============================] - 38s 4us/step - loss: 0.5191 - acc: 0.7380


0.8304535439981965

In [21]:
# Begin Attempt 2
# Use original architecture (2 hidden layers in total) but now set all layers to have 25 neurons
# Use tanh activation
model3 = Sequential()
model3.add(Dense(25, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model3.add(Activation('tanh'))
model3.add(Dropout(0.10))
model3.add(Dense(25, kernel_initializer='uniform'))
model3.add(Activation('tanh'))
model3.add(Dropout(0.10))
model3.add(Dense(1, kernel_initializer='uniform')) 
model3.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model3.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model3.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model3.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.6011 - acc: 0.6721
Epoch 2/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5710 - acc: 0.6994
Epoch 3/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5613 - acc: 0.7069
Epoch 4/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5559 - acc: 0.7108
Epoch 5/5
10500000/10500000 [==============================] - 29s 3us/step - loss: 0.5530 - acc: 0.7129


0.8068294797927273

In [22]:
# Begin Attempt 3
# Use original architecture (2 hidden layers in total) but now set all layers to have 10 neurons
# Use relu
model4 = Sequential()
model4.add(Dense(10, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model4.add(Activation('relu'))
model4.add(Dropout(0.10))
model4.add(Dense(10, kernel_initializer='uniform'))
model4.add(Activation('relu'))
model4.add(Dropout(0.10))
model4.add(Dense(1, kernel_initializer='uniform')) 
model4.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model4.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model4.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model4.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 26s 2us/step - loss: 0.5914 - acc: 0.6834
Epoch 2/5
10500000/10500000 [==============================] - 26s 2us/step - loss: 0.5760 - acc: 0.6986
Epoch 3/5
10500000/10500000 [==============================] - 25s 2us/step - loss: 0.5735 - acc: 0.6997
Epoch 4/5
10500000/10500000 [==============================] - 26s 2us/step - loss: 0.5722 - acc: 0.7003
Epoch 5/5
10500000/10500000 [==============================] - 26s 2us/step - loss: 0.5712 - acc: 0.7007


0.7884435273166096

## Work 3
Take your best model from parts 1&2 and vary the batch size by at least 2 orders of magnitude

Note from KJ:

> I tried using batch_size=10 but it took forever to run (5 minutes per epoch). The following is batch_size=10000

In [23]:
# Begin Attempt 1
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
# Batch size originally 1000, now make batch size 100000
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1, kernel_initializer='uniform')) 
model2.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model2.fit(x, y, epochs=5, batch_size=100000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 44s 4us/step - loss: 0.6914 - acc: 0.5299
Epoch 2/5
10500000/10500000 [==============================] - 43s 4us/step - loss: 0.6913 - acc: 0.5300
Epoch 3/5
10500000/10500000 [==============================] - 44s 4us/step - loss: 0.6913 - acc: 0.5300
Epoch 4/5
10500000/10500000 [==============================] - 43s 4us/step - loss: 0.6912 - acc: 0.5300
Epoch 5/5
10500000/10500000 [==============================] - 46s 4us/step - loss: 0.6897 - acc: 0.5312


0.6159894554889519

## Work 4

Take your best model (score) from parts 1&2 and use 3 different kernel initializers. Use a reasonable batch size.

KJ Note

> results are about the same when removing the kernel_initializer from subsequent layers as shown in the first attempt here

In [24]:
# Begin Attempt 1
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1)) 
model2.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 43s 4us/step - loss: 0.5587 - acc: 0.7087
Epoch 2/5
10500000/10500000 [==============================] - 43s 4us/step - loss: 0.5338 - acc: 0.7275
Epoch 3/5
10500000/10500000 [==============================] - 46s 4us/step - loss: 0.5280 - acc: 0.7318
Epoch 4/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.5248 - acc: 0.7339
Epoch 5/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5225 - acc: 0.7354


0.8288583828377185

In [25]:
# Begin Attempt 2
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
# now use lecun_normal for initialization
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='lecun_normal')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1)) 
model2.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5664 - acc: 0.7020
Epoch 2/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5369 - acc: 0.7255
Epoch 3/5
10500000/10500000 [==============================] - 38s 4us/step - loss: 0.5297 - acc: 0.7306
Epoch 4/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5261 - acc: 0.7331
Epoch 5/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5236 - acc: 0.7348


0.8290900525512799

In [26]:
# Begin Attempt 3
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
# now use random_uniform for initialization
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='random_uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1)) 
model2.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5590 - acc: 0.7084
Epoch 2/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5336 - acc: 0.7275
Epoch 3/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5279 - acc: 0.7318
Epoch 4/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5249 - acc: 0.7338
Epoch 5/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5230 - acc: 0.7351


0.8277445328276265

## Work 5

Take your best results from #3 and try 3 different optimizers. (LMGTFY)

> RMSprop, Adamax, Adagrad

In [27]:
# Begin Attempt 1
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
# Now use RMSprop
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1, kernel_initializer='uniform')) 
model2.add(Activation('sigmoid'))

optimizer=RMSprop()
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizer)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5714 - acc: 0.6977
Epoch 2/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5378 - acc: 0.7245
Epoch 3/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5292 - acc: 0.7306
Epoch 4/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5253 - acc: 0.7333
Epoch 5/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5232 - acc: 0.7347


0.8271407972356888

In [28]:
# Begin Attempt 1
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
# Now use Adamax
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1, kernel_initializer='uniform')) 
model2.add(Activation('sigmoid'))

optimizer=Adamax()
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizer)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.5673 - acc: 0.7008
Epoch 2/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.5355 - acc: 0.7267
Epoch 3/5
10500000/10500000 [==============================] - 40s 4us/step - loss: 0.5279 - acc: 0.7320
Epoch 4/5
10500000/10500000 [==============================] - 45s 4us/step - loss: 0.5240 - acc: 0.7347
Epoch 5/5
10500000/10500000 [==============================] - 43s 4us/step - loss: 0.5218 - acc: 0.7360


0.8286814486563112

In [29]:
# Begin Attempt 1
# Add a layer (3 hidden layers in total) with 50 neurons
# Now use a relu
# Now use Adamax
model2 = Sequential()
model2.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(50, kernel_initializer='uniform'))
model2.add(Activation('relu'))
model2.add(Dropout(0.10))
model2.add(Dense(1, kernel_initializer='uniform')) 
model2.add(Activation('sigmoid'))

optimizer=Adagrad()
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizer)

model2.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model2.predict(x_test))

Epoch 1/5
10500000/10500000 [==============================] - 43s 4us/step - loss: 0.5818 - acc: 0.6899
Epoch 2/5
10500000/10500000 [==============================] - 43s 4us/step - loss: 0.5555 - acc: 0.7124
Epoch 3/5
10500000/10500000 [==============================] - 42s 4us/step - loss: 0.5468 - acc: 0.7186
Epoch 4/5
10500000/10500000 [==============================] - 39s 4us/step - loss: 0.5413 - acc: 0.7226
Epoch 5/5
10500000/10500000 [==============================] - 41s 4us/step - loss: 0.5373 - acc: 0.7251


0.8148219277373501

## Work 6

Take all that you’ve learned so far and give your best shot at producing a score.

#Begin 
# 1 layer (prof code)
model1 = Sequential()
model1.add(Dense(50, input_dim=x.shape[1], kernel_initializer='uniform')) # X_train.shape[1] == 28 here
model1.add(Activation('relu'))
model1.add(Dropout(0.10))
model1.add(Dense(50, kernel_initializer='uniform'))
model1.add(Activation('relu'))
model1.add(Dropout(0.10))
model1.add(Dense(1, kernel_initializer='uniform')) 
model1.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)

model1.fit(x, y, epochs=5, batch_size=1000)
roc_auc_score(y_test,model1.predict(x_test))
#end